## Laboratorio 4 - Generación de columnas

Integrantes:
- Sebastián Gangas
- Marco Ipinza
- Matías Nuñez
- Ignacio Díaz

In [1]:
# Imports
using JuMP, Gurobi, DataFrames
const GUROBI_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26


Gurobi.Env(Ptr{Nothing} @0x0000022ac9f105e0, false, 0)

### Ejercicio 1

In [2]:
function red(nombrearchivo)
    archivo = open(nombrearchivo)
    lineas = readlines(archivo)
    l = length(lineas)
    V,k = parse.(Int32, split(lineas[1], ' '))
    s = zeros((k))
    t = zeros((k))
    d = zeros((k))
    for i in 1:(k)
        si,ti,di = parse.(Int32, split(lineas[i+1], ' '))
        s[i] = si
        t[i] = ti 
        d[i] = di
    end
    E = []
    c = zeros((V,V))
    u = zeros((V,V))
    for z in (k+2):l
        linea = lineas[z]
        i, j, cij, uij = parse.(Int32, split(linea, ' '))
        push!(E, [i,j])
        c[i,j] = cij
        u[i,j] = uij
    end
    return (V, k, s, t, d, E, c, u)
end

red (generic function with 1 method)

### Ejercicio 2

In [3]:
function modelo_MCF(V, K, s, t, d, E, c, u, warm, entero)
    mimodelo = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"Presolve"=> 0, "OutputFlag" => 0))
    #habria que agregar el a dentro del input supongo
    if entero
        #if warm 
        #    @variable(mimodelo, 0<=x[i in 1:V, j in 1:V, k in 1:K], Int, start = a[i,j,k])
        #else
            @variable(mimodelo, 0<=x[i in 1:V, j in 1:V, k in 1:K], Int)
        #end
    else
        #if warm
            @variable(mimodelo, 0<=x[i in 1:V, j in 1:V, k in 1:K], start = a[i,j,k])
        #else
            @variable(mimodelo, 0<=x[i in 1:V, j in 1:V, k in 1:K], start = a[i,j,k])   
        #end
    end

    @objective(mimodelo, Min, sum(c[i,j]*sum(x[i,j,k] for k in 1:K) for i in 1:V for j in 1:V))

    @constraint(mimodelo, capacidad[i in 1:V, j in 1:V], sum(x[i,j,k] for k in 1:K)<=u[i,j])
    @constraint(mimodelo, flujo[i in 1:V, k in 1:K], sum(x[i,j,k] for j in 1:V) - sum(x[j,i,k] for j in 1:V) == d[k]*(i==s[k])-d[k]*(i==t[k]))
    return mimodelo
end

modelo_MCF (generic function with 1 method)

In [3]:
# en lo anterior hay que ver bien el tema de cuando pasamos a dantzig wolfe si los caminos están considerando a wnes inexistentes 
# principalmente xq la matriz cuadrada nos da más elementos (que debieran tener capacidad 0) y dps no ser considerados como st camino

modelo_MCF (generic function with 1 method)

In [4]:
function df_solucion(x)
    I,J,K = size(x)
    flujo = []
    E = []
    d = []
    for i in 1:I
        for j in 1:J
            if !([i,j] in E)
                s = []
                push!(E,[i,j])
                for k in 1:K
                    push!(s,x[i,j,k])
                end
                push!(d, s)
            end
        end
    end
    l = length(E)
    M = zeros((k+2,l))
    for i in 1:l
        M[1,i] = E[i][1]
        M[2,i] = E[i][2]
        for k in 1:K
            M[k,i] = d[i][k]
        end
    end
    i = M[1,:]
    j = M[2,:]
    df = DataFrame()
    df.inicio = i
    df.fin = j
    for k in 1:K
        df[!,:k] = M[k+2,:]
    end
    print(df)
end 

df_solucion (generic function with 1 method)

In [ ]:
#en la descomposición luego se ignoran los z_c,k porque estos representan caminos por ciclos, pero para llegar a estar en un óptimo si este ocupa ciclos
# no debiera ser óptimo (se demuestra por contradicción: se asume que si, notar que los ciclos agregan costo (se justifica con que como la función de
# costos por arista es real positiva, no existen ciclos con costo negativo), y como son ciclos vuelven a donde mismo
# partieron [si no es el caso entonces ese tramo no era un ciclo, y este está en otro lugar aunque debiera bastar decir que por definición es eso nms])
# ahora como vuelven al mismo punto y luego se encaminan hacia el t_k tenemos que podemos tomar el st_k camino que no considera el ciclo: con lo cual hay 2 casos
# el costo de z_c,k es >0 (con lo cual obtubimos un costo menor contradicción con que es óptimo). el costo es = 0, con ello tenemos que el 
# tomar el ciclo o no no impacta al costo minimal, con lo cual podemos no considerarlo y aún así estamos en un óptimo 